In [32]:
%pip install huggingface_hub openai vllm llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 37.9 MB/s  0:00:01 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-macosx_15_0_arm64.whl size=3848476 sha256=452fc645b012c36d7e9016fc56a96a8526d509cc520f82253b754449ba6a69f3
  Stored in directory: /Users/danielwohlgemuth/Library/Caches/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python
Note: you may need to restart the kernel to use updated packages.


## TGI

Launch the TGI server using Docker. Note: There is no support for Macbook M series.

```bash
docker run --gpus all \
    --shm-size 1g \
    -p 8080:80 \
    -v ~/.cache/huggingface:/data\
    ghcr.io/huggingface/text-generation-inference:latest \
    --model-id HuggingFaceTB/SmolLM2-360M-Instruct \
    --device 0
```

Interact with TGI using the Hugging Face Inference Client

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    model="http://localhost:8080",
)

response = client.text_generation(
    "Tell me a story",
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.95,
    details=True,
    stop_sequences=[],
)
print(response.generated_text)

response = client.chat_completion(
    messages=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": "Tell me a story" },
    ],
    max_tokens=100,
    temperature=0.7,
    top_p=0.95,
)
print(response.choices[0].message.content)

Interact with TGI using the OpenAI client

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8080",
    api_key="not-needed",
)

response = client.chat.completion.create(
    model="HuggingFaceTB/SmolLM2-360M-Instruct",
    message=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": "Tell a story" },
    ],
    max_tokens=100,
    temperature=0.7,
    top_p=0.95,
)
print(response.choices[0].messages.content)

## llama.cpp

In [ ]:
git clone https://github.com/ggerganov/llama.cpp --depth 1
cd llama.cpp

cmake -B build
cmake --build build --config Release -j 8

curl -L -O "https://huggingface.co/HuggingFaceTB/SmolLM2-1.7B-Instruct-GGUF/resolve/main/smollm2-1.7b-instruct-q4_k_m.gguf?download=true" -o ./models/smollm2-1.7b-instruct-q4_k_m.gguf

./build/bin/llama-server \
    -m ./models/smollm2-1.7b-instruct-q4_k_m.gguf \
    --host 0.0.0.0 \
    --port 8080 \
    -c 4096 \
    --n-gpu-layers 0

Alternative using Docker

In [ ]:
docker run -v ./models:/models -p 8080:8080 ghcr.io/ggml-org/llama.cpp:server -m /models/smollm2-1.7b-instruct-q4_k_m.gguf --port 8080 --host 0.0.0.0 -n 512

In [16]:
from huggingface_hub import InferenceClient

# Initialize client pointing to llama.cpp server
client = InferenceClient(
    model="http://localhost:8080/v1",  # URL to the llama.cpp server
    token="sk-no-key-required",  # llama.cpp server requires this placeholder
)

# Text generation
# response = client.text_generation(
#     "Tell me a story",
#     max_new_tokens=100,
#     temperature=0.7,
#     top_p=0.95,
#     details=True,
# )
# print(response.generated_text)

# For chat format
response = client.chat_completion(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a story"},
    ],
    max_tokens=100,
    temperature=0.7,
    top_p=0.95,
)
print(response.choices[0].message.content)

In the land of Aethoria, where the sun dipped into the horizon and painted the sky with hues of crimson and gold, there was a village known for its magical creatures. Among them was a young fawn named Luna, who possessed an extraordinary gift – she was the only one in Aethoria who could communicate with the magical animals.

Luna lived with her wise old mother, a talking owl named Orion, in a cozy little hut surrounded by a lush forest filled with the


In [13]:
from openai import OpenAI

# Initialize client pointing to llama.cpp server
client = OpenAI(
    base_url="http://localhost:8080/v1",
    api_key="sk-no-key-required",  # llama.cpp server requires this placeholder
)

# Chat completion
response = client.chat.completions.create(
    model="smollm2-1.7b-instruct",  # Model identifier can be anything as server only loads one model
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a story"},
    ],
    max_tokens=100,
    temperature=0.7,
    top_p=0.95,
)
print(response.choices[0].message.content)

Once upon a time, in a world of endless possibilities, there was a young girl named Luna. She was a curious and adventurous girl, always eager to explore the world around her. She lived in a small village at the edge of a vast and mysterious forest.

One day, Luna decided that she wanted to explore the forest. She packed her bag with food, water, and a map of the forest. Her parents were a bit worried, but they trusted her to be safe.



## vLLM

In [ ]:
python3 -m venv venv
source venv/bin/activate
pip install vllm
python3 -m vllm.entrypoints.openai.api_server \
    --model HuggingFaceTB/SmolLM2-360M-Instruct \
    --host 0.0.0.0 \
    --port 8000

In [26]:
from huggingface_hub import InferenceClient

# Initialize client pointing to vLLM endpoint
client = InferenceClient(
    model="http://localhost:8000/v1",  # URL to the vLLM server
)

# Text generation
# response = client.text_generation(
#     "Tell me a story",
#     max_new_tokens=100,
#     temperature=0.7,
#     top_p=0.95,
#     details=True,
# )
# print(response.generated_text)

# For chat format
response = client.chat_completion(
    model="HuggingFaceTB/SmolLM2-360M-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a story"},
    ],
    max_tokens=100,
    temperature=0.7,
    top_p=0.95,
)
print(response.choices[0].message.content)

Once upon a time, in a faraway land called Azura, there lived a wise and powerful wizard named Eryndor Thorne. Eryndor was a master of ancient magic and had spent his life studying the mystical forces of the universe. He was known for his extraordinary abilities and was often sought after by those who sought to learn from his vast knowledge.

One day, Eryndor was summoned by the kingdom of Everia, a land of beautiful gardens and prosperous


In [27]:
from openai import OpenAI

# Initialize client pointing to vLLM endpoint
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="not-needed",  # vLLM doesn't require an API key by default
)

# Chat completion
response = client.chat.completions.create(
    model="HuggingFaceTB/SmolLM2-360M-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a story"},
    ],
    max_tokens=100,
    temperature=0.7,
    top_p=0.95,
)
print(response.choices[0].message.content)

Once upon a time, in a land far away, there lived a wise and powerful wizard named Thorold. Thorold was the king of the land, and his power was feared by all who lived there. He had a staff that could read the thoughts of those around him and could cast spells of destruction.

One day, Thorold was at a great feast in the castle. He was surrounded by a group of his most trusted advisors, including his closest friend and confidant,


TGI with advanced paramters

In [ ]:
docker run --gpus all \
    --shm-size 1g \
    -p 8080:80 \
    -v ~/.cache/huggingface:/data \
    ghcr.io/huggingface/text-generation-inference:latest \
    --model-id HuggingFaceTB/SmolLM2-360M-Instruct \
    --max-total-tokens 4096 \
    --max-input-length 3072 \
    --max-batch-total-tokens 8192 \
    --waiting-served-ratio 1.2

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(model="http://localhost:8080")

# Advanced parameters example
response = client.chat_completion(
    messages=[
        {"role": "system", "content": "You are a creative storyteller."},
        {"role": "user", "content": "Write a creative story"},
    ],
    temperature=0.8,
    max_tokens=200,
    top_p=0.95,
)
print(response.choices[0].message.content)

# Raw text generation
response = client.text_generation(
    "Write a creative story about space exploration",
    max_new_tokens=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.1,
    do_sample=True,
    details=True,
)
print(response.generated_text)

In [ ]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8080/v1", api_key="not-needed")

# Advanced parameters example
response = client.chat.completions.create(
    model="HuggingFaceTB/SmolLM2-360M-Instruct",
    messages=[
        {"role": "system", "content": "You are a creative storyteller."},
        {"role": "user", "content": "Write a creative story"},
    ],
    temperature=0.8,  # Higher for more creativity
)
print(response.choices[0].message.content)

llama.cpp with advanced parameters

In [ ]:
./build/bin/llama-server \
    -m ./models/smollm2-1.7b-instruct-q4_k_m.gguf \
    --host 0.0.0.0 \
    --port 8080 \
    -c 4096 \
    --n-gpu-layers 0

In [29]:
from huggingface_hub import InferenceClient

client = InferenceClient(model="http://localhost:8080/v1", token="sk-no-key-required")

# Advanced parameters example
response = client.chat_completion(
    messages=[
        {"role": "system", "content": "You are a creative storyteller."},
        {"role": "user", "content": "Write a creative story"},
    ],
    temperature=0.8,
    max_tokens=200,
    top_p=0.95,
)
print(response.choices[0].message.content)

# For direct text generation
# response = client.text_generation(
#     "Write a creative story about space exploration",
#     max_new_tokens=200,
#     temperature=0.8,
#     top_p=0.95,
#     repetition_penalty=1.1,
#     details=True,
# )
# print(response.generated_text)

Once upon a time, in a far-off land, there lived a young girl named Luna. She was an adventurous girl, always seeking new experiences and learning from her surroundings. Luna lived in a quaint little village by the sea, where the salty winds and the salty sea air filled her lungs, and the salty sea water danced on her skin.

One day, while Luna was out on her favorite spot on the beach, a sudden gust of wind swept in and blew her golden-blonde hair into a tangled mess. Luna was frantically trying to untangle her hair, but it seemed impossible. She was so frustrated that she decided to go home and get a hair cut, but before she left, she decided to ask her grandmother for help.

"Grandma, can you help me untangle my hair?" Luna asked, her voice filled with excitement.

Her grandmother, an old lady with a kind heart and a beautiful silver hair, smiled and nodded. She


In [ ]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8080/v1", api_key="sk-no-key-required")

# Advanced parameters example
response = client.chat.completions.create(
    model="smollm2-1.7b-instruct",
    messages=[
        {"role": "system", "content": "You are a creative storyteller."},
        {"role": "user", "content": "Write a creative story"},
    ],
    temperature=0.8,  # Higher for more creativity
    top_p=0.95,  # Nucleus sampling probability
    frequency_penalty=0.5,  # Reduce repetition of frequent tokens
    presence_penalty=0.5,  # Reduce repetition by penalizing tokens already present
    max_tokens=200,  # Maximum generation length
)
print(response.choices[0].message.content)

Once upon a time, in the land of Fantasia, there was a mystical creature named Luna. She was a being with the essence of both moon and stars within her. Luna was known for her enchanting beauty and divine powers that she used to bring peace to the warring creatures of Fantasia.

Luna had an extraordinary ability - she could transform into any celestial body or any shape she wished, which made her one of the most feared as well as revered beings in Fantasia. She could morph into a shooting star, a bright moonlight, a radiant sunbeam or even a majestic starry sky at will.

Luna was adored by all the creatures for her enchanting beauty and divine powers. They believed that she had magical powers to heal any ailment, bring good fortune and prosperity, and even grant wishes. Luna was kind-hearted and always used her powers for the good of the world.

One day, a wicked witch named Malice sought to capture Luna


In [31]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8080/v1", api_key="sk-no-key-required")

# Advanced parameters example
response = client.chat.completions.create(
    model="smollm2-1.7b-instruct",
    messages=[
        {"role": "system", "content": "You are a creative storyteller."},
        {"role": "user", "content": "Write a creative story"},
    ],
    temperature=0.1,  # Higher for more creativity
    top_p=0.95,  # Nucleus sampling probability
    frequency_penalty=0.9,  # Reduce repetition of frequent tokens
    presence_penalty=0.9,  # Reduce repetition by penalizing tokens already present
    max_tokens=100,  # Maximum generation length
)
print(response.choices[0].message.content)

# Advanced parameters example
response = client.chat.completions.create(
    model="smollm2-1.7b-instruct",
    messages=[
        {"role": "system", "content": "You are a creative storyteller."},
        {"role": "user", "content": "Write a creative story"},
    ],
    temperature=0.1,  # Higher for more creativity
    top_p=0.95,  # Nucleus sampling probability
    frequency_penalty=0.1,  # Reduce repetition of frequent tokens
    presence_penalty=0.1,  # Reduce repetition by penalizing tokens already present
    max_tokens=100,  # Maximum generation length
)
print(response.choices[0].message.content)

Once upon a time, in the land of Imagination, there was an enchanted forest filled with magical creatures and talking animals. Among them were two best friends, Luna the Luminous Fairy and Orion the Orangutan. They lived in harmony, exploring every nook and cranny of their enchanting home.

One day, while wandering through a particularly dense part of the forest, they stumbled upon an ancient tree with glowing leaves that shimmered like stars in the night sky. Luna was immediately
Once upon a time, in a land far, far away, there was a magical kingdom known as the Land of Whimsy. This kingdom was filled with fantastical creatures, enchanted forests, and hidden treasures. Among the most magical of all the creatures in this kingdom was a young girl named Luna. Luna was a dreamer, always chasing her dreams and always believing in the magic that lay just beyond the edge of reality.

Luna lived in a small cottage on the outskirts of the


Using llama.cpp's Python library

In [33]:
from llama_cpp import Llama

llm = Llama(
    model_path="llama.cpp/models/smollm2-1.7b-instruct-q4_k_m.gguf",
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=0,
)

prompt = """<|im_start|>system
You are a creative storyteller.
<|im_end|>
<|im_start|>user
Write a creative story
<|im_end|>
<|im_start|>assistant
"""

output = llm(
    prompt,
    max_tokens=200,
    temperature=0.8,
    top_p=0.95,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    stop=["<|im_end|>"],
)

print(output["choices"][0]["text"])


llama_model_load_from_file_impl: using device Metal (Apple M3) - 10922 MiB free
llama_model_loader: loaded meta data with 34 key-value pairs and 218 tensors from llama.cpp/models/smollm2-1.7b-instruct-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Smollm2 1.7B 8k Mix7 Ep2 v2
llama_model_loader: - kv   3:                            general.version str              = v2
llama_model_loader: - kv   4:                       general.organization str              = Loubnabnl
llama_model_loader: - kv   5:                           general.finetune str              = 8k-mix7-ep2
llama_model_loader: - kv   6:                

In a world where time was currency, the rich could live forever and the poor were left with nothing but the tick-tock of their mortality. A young man named Jack was born with an unusual gift – he could manipulate time. With his powers, Jack could slow down or speed up time to suit his needs.

One day, while working at a small clock repair shop, Jack's life changed forever when a wealthy businessman walked in and asked for his help fixing an antique pocket watch. To Jack's surprise, the watch was priceless and held a secret. It was created by a mysterious timekeeper who had imbued it with the ability to manipulate time itself.

As Jack worked on the watch, he began to realize that he could use its power to change his life forever. He could speed up time when he needed it, slowing down the days until their arrival like a gentle breeze on a summer's day. And when his life became too slow, he could speed it up,


vLLM advanced parameters

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(model="http://localhost:8000/v1")

# Advanced parameters example
response = client.chat_completion(
    messages=[
        {"role": "system", "content": "You are a creative storyteller."},
        {"role": "user", "content": "Write a creative story"},
    ],
    temperature=0.8,
    max_tokens=200,
    top_p=0.95,
)
print(response.choices[0].message.content)

# For direct text generation
response = client.text_generation(
    "Write a creative story about space exploration",
    max_new_tokens=200,
    temperature=0.8,
    top_p=0.95,
    details=True,
)
print(response.generated_text)

In [ ]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8000/v1", api_key="not-needed")

# Advanced parameters example
response = client.chat.completions.create(
    model="HuggingFaceTB/SmolLM2-360M-Instruct",
    messages=[
        {"role": "system", "content": "You are a creative storyteller."},
        {"role": "user", "content": "Write a creative story"},
    ],
    temperature=0.8,
    top_p=0.95,
    max_tokens=200,
)
print(response.choices[0].message.content)

Using vLLM's Python library

In [37]:
from vllm import LLM, SamplingParams

# Initialize the model with advanced parameters
llm = LLM(
    model="HuggingFaceTB/SmolLM2-360M-Instruct",
    gpu_memory_utilization=0.85,
    max_num_batched_tokens=8192,
    max_num_seqs=256,
    block_size=16,
)

# Configure sampling parameters
sampling_params = SamplingParams(
    temperature=0.8,  # Higher for more creativity
    top_p=0.95,  # Consider top 95% probability mass
    max_tokens=100,  # Maximum length
    presence_penalty=1.1,  # Reduce repetition
    frequency_penalty=1.1,  # Reduce repetition
    stop=["\n\n", "###"],  # Stop sequences
)

# Generate text
prompt = "Write a creative story"
outputs = llm.generate(prompt, sampling_params)
print(outputs[0].outputs[0].text)

# For chat-style interactions
chat_prompt = [
    {"role": "system", "content": "You are a creative storyteller."},
    {"role": "user", "content": "Write a creative story"},
]
formatted_prompt = llm.preprocess_chat(chat_prompt)  # Uses model's chat template
outputs = llm.generate(formatted_prompt, sampling_params)
print(outputs[0].outputs[0].text)

INFO 02-08 16:23:10 [utils.py:263] non-default args: {'block_size': 16, 'gpu_memory_utilization': 0.85, 'max_num_batched_tokens': 8192, 'max_num_seqs': 256, 'disable_log_stats': True, 'model': 'HuggingFaceTB/SmolLM2-360M-Instruct'}
INFO 02-08 16:23:11 [model.py:530] Resolved architecture: LlamaForCausalLM
INFO 02-08 16:23:11 [model.py:1545] Using max model len 8192
INFO 02-08 16:23:11 [scheduler.py:229] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 02-08 16:23:11 [cpu.py:190] CPU backend prefers block_size is multiples of 32, otherwise the performance is not optimized.
INFO 02-08 16:23:13 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.
(EngineCore_DP0 pid=25878) INFO 02-08 16:23:14 [core.py:97] Initializing a V1 LLM engine (v0.14.1) with config: model='HuggingFaceTB/SmolLM2-360M-Instruct', speculative_config=None, tokenizer='HuggingFaceTB/SmolLM2-360M-Instruct', skip_tokenizer_init=False, tokenizer_mo

[W208 16:23:20.158669000 ProcessGroupGloo.cpp:547] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


(EngineCore_DP0 pid=25878) INFO 02-08 16:24:20 [parallel_state.py:1425] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, PCP rank 0, TP rank 0, EP rank N/A
(EngineCore_DP0 pid=25878) INFO 02-08 16:24:20 [cpu_model_runner.py:55] Starting to load model HuggingFaceTB/SmolLM2-360M-Instruct...
(EngineCore_DP0 pid=25878) INFO 02-08 16:24:21 [weight_utils.py:550] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.13it/s]
(EngineCore_DP0 pid=25878) 


(EngineCore_DP0 pid=25878) INFO 02-08 16:24:22 [default_loader.py:291] Loading weights took 0.89 seconds
(EngineCore_DP0 pid=25878) INFO 02-08 16:24:22 [kv_cache_utils.py:1305] GPU KV cache size: 209,712 tokens
(EngineCore_DP0 pid=25878) INFO 02-08 16:24:22 [kv_cache_utils.py:1310] Maximum concurrency for 8,192 tokens per request: 25.60x
(EngineCore_DP0 pid=25878) INFO 02-08 16:24:23 [cpu_model_runner.py:65] Warming up model for the compilation...
(EngineCore_DP0 pid=25878) INFO 02-08 16:24:28 [cpu_model_runner.py:75] Warming up done.
(EngineCore_DP0 pid=25878) INFO 02-08 16:24:28 [core.py:273] init engine (profile, create kv cache, warmup model) took 6.61 seconds
(EngineCore_DP0 pid=25878) INFO 02-08 16:24:29 [vllm.py:630] Asynchronous scheduling is disabled.
(EngineCore_DP0 pid=25878) WARNING 02-08 16:24:29 [vllm.py:672] Inductor compilation was disabled by user settings, optimizations settings that are only active during inductor compilation will be ignored.
(EngineCore_DP0 pid=2587

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.69s/it, est. speed input: 2.37 toks/s, output: 49.69 toks/s]


 idea for a young female detective character with empathy and compassion. Your story should include:
1. A relatable protagonist with a unique backstory,
2. A compelling crime mystery to solve,
3. Engaging dialogue that showcases her emotional intelligence and interpersonal skills, and
4. An exciting plot twist that will captivate readers' attention and encourage them to continue reading to find out what happens next in the story.
INFO 02-08 16:24:33 [chat_utils.py:599] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it, est. speed input: 19.93 toks/s, output: 47.04 toks/s]

In a small, seaside town lived a curious cat named Luna. Luna was known for her incredible ability to solve mysteries and figure out puzzles that others found impossible. One sunny day, while wandering around the beach, she stumbled upon an old wooden box hidden beneath a pile of sand.


## Advanced Generation Control

### Token Selection & Sampling

TGI

In [ ]:
client.generate(
    "Write a creative story",
    temperature=0.8,  # Higher for more creativity
    top_p=0.95,  # Consider top 95% probability mass
    top_k=50,  # Consider top 50 tokens
    max_new_tokens=100,  # Maximum length
    repetition_penalty=1.1,  # Reduce repetition
)

llama.cpp

In [ ]:
# Via OpenAI API compatibility
response = client.completions.create(
    model="smollm2-1.7b-instruct",  # Model name (can be any string for llama.cpp server)
    prompt="Write a creative story",
    temperature=0.8,  # Higher for more creativity
    top_p=0.95,  # Consider top 95% probability mass
    frequency_penalty=1.1,  # Reduce repetition
    presence_penalty=0.1,  # Reduce repetition
    max_tokens=100,  # Maximum length
)

# Via llama-cpp-python direct access
output = llm(
    "Write a creative story",
    temperature=0.8,
    top_p=0.95,
    top_k=50,
    max_tokens=100,
    repeat_penalty=1.1,
)

vLLM

In [ ]:
params = SamplingParams(
    temperature=0.8,  # Higher for more creativity
    top_p=0.95,  # Consider top 95% probability mass
    top_k=50,  # Consider top 50 tokens
    max_tokens=100,  # Maximum length
    presence_penalty=0.1,  # Reduce repetition
)
llm.generate("Write a creative story", sampling_params=params)

### Control Repetition

TGI

In [ ]:
client.generate(
    "Write a varied text",
    repetition_penalty=1.1,  # Penalize repeated tokens
    no_repeat_ngram_size=3,  # Prevent 3-gram repetition
)

llama.cpp

In [ ]:
# Via OpenAI API
response = client.completions.create(
    model="smollm2-1.7b-instruct",
    prompt="Write a varied text",
    frequency_penalty=1.1,  # Penalize frequent tokens
    presence_penalty=0.8,  # Penalize tokens already present
)

# Via direct library
output = llm(
    "Write a varied text",
    repeat_penalty=1.1,  # Penalize repeated tokens
    frequency_penalty=0.5,  # Additional frequency penalty
    presence_penalty=0.5,  # Additional presence penalty
)

vllm

In [ ]:
params = SamplingParams(
    presence_penalty=0.1,  # Penalize token presence
    frequency_penalty=0.1,  # Penalize token frequency
)

### Length Control and Stop Sequences

TGI

In [ ]:
client.generate(
    "Generate a short paragraph",
    max_new_tokens=100,
    min_new_tokens=10,
    stop_sequences=["\n\n", "###"],
)

llama.cpp

In [ ]:
# Via OpenAI API
response = client.completions.create(
    model="smollm2-1.7b-instruct",
    prompt="Generate a short paragraph",
    max_tokens=100,
    stop=["\n\n", "###"],
)

# Via direct library
output = llm("Generate a short paragraph", max_tokens=100, stop=["\n\n", "###"])

vllm

In [ ]:
params = SamplingParams(
    max_tokens=100,
    min_tokens=10,
    stop=["###", "\n\n"],
    ignore_eos=False,
    skip_special_tokens=True,
)

### Memory Management

TGI

In [ ]:
# Docker deployment with memory optimization
docker run --gpus all -p 8080:80 \
    --shm-size 1g \
    ghcr.io/huggingface/text-generation-inference:latest \
    --model-id HuggingFaceTB/SmolLM2-1.7B-Instruct \
    --max-batch-total-tokens 8192 \
    --max-input-length 4096

llama.cpp

In [ ]:
# Server with memory optimizations
./server \
    -m smollm2-1.7b-instruct.Q4_K_M.gguf \
    --host 0.0.0.0 \
    --port 8080 \
    -c 2048 \               # Context size
    --threads 4 \           # CPU threads
    --n-gpu-layers 32 \     # Use more GPU layers for larger models
    --mlock \               # Lock memory to prevent swapping
    --cont-batching         # Enable continuous batching

In [ ]:
./server \
    -m smollm2-1.7b-instruct.Q4_K_M.gguf \
    --n-gpu-layers 20 \     # Keep first 20 layers on GPU
    --threads 8             # Use more CPU threads for CPU layers

vLLM

In [ ]:
from vllm.engine.arg_utils import AsyncEngineArgs

engine_args = AsyncEngineArgs(
    model="HuggingFaceTB/SmolLM2-1.7B-Instruct",
    gpu_memory_utilization=0.85,
    max_num_batched_tokens=8192,
    block_size=16,
)

llm = LLM(engine_args=engine_args)